MCDA 코드 분석

In [19]:
#env 로드
import os
import topogenesis as tg
import trimesh as tm
import numpy as np
import pyvista as pv
import panel as pn
import pandas as pd 

from ladybug.sunpath import Sunpath #ladybug sunpath 호출
from IPython.display import display, Image

In [20]:
# csv에서 lattice 배열 만드는 함수 정의
def lattice_from_csv(file_path):
    # 데이터를 읽어옴
    meta_df = pd.read_csv(file_path, nrows=3)
    # shape, unit, minbound 데이터를 배열로 변환
    shape = np.array(meta_df['shape']) # 시작 좌표 [x_min, y_min, z_min] 예를 들어, [-55000, -7000, 0] x축 -55000, y축 -7000, z축 0의 위치
    unit = np.array(meta_df['unit'])   # 몇개의 셀로 구성되는지 [x_shape, y_shape, z_shape] 예를 들어 [56, 35, 24]이면 x축 56개, y 35개, z 24개의 셀을 가집니다.
    minbound = np.array(meta_df['minbound']) # 각 셀의 크기. 예를 들어, [1000, 1000, 3000] 각 셀의 크기는 x축 1000 단위, y축 1000 단위, z축 3000 단위
    # np.array는 csv에서 불러온 데이터를 배열로 변환하는데 사용 
    # 그리드 데이터를 읽어옴
    lattice_df = pd.read_csv(file_path, skiprows=5) # 앞의 5개 줄을 스킵하고 6개 부터 데이터를 불러오기
    # 데이터의 'value' 열을 배열로 변환한 후 이를 'shape' 배열에 맞게 재구성 
    buffer = np.array(lattice_df['value']).reshape(shape)
    # lattice를 토포제네시스에서 재구성 buffer는 배열에 맞는 value값. 
    l = tg.to_lattice(buffer, minbound=minbound, unit=unit)
    
    return l

In [21]:
# tri_to_pv 함수 정의: 삼각형 메시(tri_mesh)를 PyVista 형식으로 변환합니다.
def tri_to_pv(tri_mesh):
    #  tri_mesh.faces 배열에 각 face의 시작 부분에 3을 추가하여 새로운 배열을 만듭니다.
    # 'constant' 모드로 패딩을 추가하여 각 face의 시작에 3을 삽입합니다.
    faces = np.pad(tri_mesh.faces, ((0, 0), (1, 0)), 'constant', constant_values=3)
    # PyVista의 PolyData 객체를 생성하여, tri_mesh의 정점(vertices)과 패딩된 faces 배열을 사용합니다.
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)    
    # 변환된 PyVista 메쉬 객체를 반환합니다.
    return pv_mesh

In [22]:
#csv 로드
csv_path = os.path.relpath('original_lattice/interior_lattice.csv')
#obj 포인트 csv 로드
context_mesh = tm.load(os.path.relpath('obj\context.obj'))  # 주변환경 obj 메시 불러오기
street_pc = tg.cloud_from_csv("pts\pts_str(11).CSV")        # street point as csv (포인트를 csv로 저장) 
entrance_pc = tg.cloud_from_csv("pts\ent_pts01.CSV")       # entrance point as csv (라이노 포인트를 csv로 저장)
envelope_lattice = tg.lattice_from_csv(csv_path)

In [23]:
avail_lattice = tg.lattice_from_csv(csv_path)                                         # CSV 파일에서 격자 데이터 불러오기
init_avail_lattice = tg.to_lattice(np.copy(avail_lattice), avail_lattice)             # 복사된 데이터를 격자 객체로 변환

### 이웃 패턴(Neighborhood Pattern) 정의

구조에서 특정 셀과 `이웃 관계`를 가지는 셀들의 집합을 정의하는 방식

- **격자(Lattice)**: 3차원 공간을 작은 셀로 나눈 구조입니다.
- **셀(Cell)**: 격자를 구성하는 가장 작은 단위입니다.
- **이웃(Neighbor)**: 특정 셀과 인접한 셀들입니다.

#### 패턴의 종류

`von Neumann 패턴`
- 셀의 상하좌우에 위치한 셀들을 이웃으로 간주합니다.
```markdown
  [ ] [ ] [ ]    [ ] [1] [ ]
  [ ] [R] [ ] => [2] [R] [3]
  [ ] [ ] [ ]    [ ] [4] [ ]
```

`Moore 패턴`
- 셀의 대각선 방향까지 포함하여 더 많은 셀들을 이웃으로 간주합니다
```markdown
  [ ] [ ] [ ]    [1] [2] [3]
  [ ] [R] [ ] => [4] [R] [5]
  [ ] [ ] [ ]    [6] [7] [8]
```



In [24]:
# 패턴1 - 1이라는 범위에 대한 이웃패턴 정의 (중심을 포함함) 
stencil = tg.create_stencil("von_neumann", 1, 1)
# 중심을 인덱스 비활성화 - [0,0,0]을 0으로 세팅함으로 중심을 패턴 비활성화
stencil.set_index([0,0,0], 0)

# 패턴2 - squareness 가로로 agent가 땅따먹을 수 있도록 이웃패턴2 정의 
stencil_sq = tg.create_stencil("von_neumann", 1, 1)
# 중심 인덱스 비활성화  
stencil_sq.set_index([0,0,0], 0)
# 인덱스 설정: y축 방향으로 1과 -1 위치를 0으로 설정하여 비활성화
stencil_sq.set_index([0,0,1], 0)
stencil_sq.set_index([0,0,-1], 0)

In [25]:
# PyVista 플로터 초기화
p = pv.Plotter(notebook=True, window_size=[600, 200])  # 가로 800, 세로 500 픽셀
# PyVista ImageData 객체 생성 (pyvista에 imagedata객체 생성)
grid = pv.ImageData()
# 배열의 차원 설정 (stencil_sq의 형태에 1을 더한 값)
grid.dimensions = np.array(stencil_sq.shape) + 1
grid.origin = [0, 0, 0]   # 배열의 시작점 설정
grid.spacing = [1, 1, 1]   # 배열의 셀 크기 설정 

# `stencil_sq는 3차원의 배열`grid 객체에 이를 저장하려면 1차원의 배열로 변환해야함

# 예를 들어 stencil_sq = np.array([[[1, 2], [3, 4]], [[5, 6], [7, 8]]])
# 포트란 스타일(order="F")로 평탄화하면: [1, 5, 3, 7, 2, 6, 4, 8]

grid.cell_data["values"] = stencil_sq.flatten(order="F")

# 배열 데이터에서 값이 0.9와 1.1 사이에 있는 셀들 `1`을 선택
threshed = grid.threshold([0.9, 1.1])
# 값을 메쉬로 추가, 가장자리 표시, 색상 및 불투명도 설정
p.add_mesh(threshed, show_edges=True, color="#ff8fa3", opacity=0.3)
p.show(jupyter_backend="html", return_viewer=True) # 플로터 표시

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

In [26]:
# 'excel/program (이노션).xlsx' 파일에서 'Sheet1' 시트를 읽어옴
df = pd.read_excel('excel/program (이노션).xlsx', sheet_name='Sheet1')

# 선택한 열을 복사하여 program_prefs 데이터프레임 생성
# [sun_acc 태양 근접 선호도], [ent_acc 입구 근접 선호도], [str_acc 도로 근접 선호도], [ung_pre 지하 선호도], [top_pre 탑층 선호도] 
program_prefs = df[['sun_acc', 'ent_acc', 'str_acc', 'ung_pre', 'top_pre']].copy()

# 인덱스를 재설정(기존 index를 버리고 새로운 인덱스로) 0부터 숫자 인덱스를 생성
program_prefs.reset_index(drop=True, inplace=True)
# program_prefs 데이터프레임 출력 (주석 처리됨)
print(program_prefs)

    sun_acc  ent_acc  str_acc  ung_pre  top_pre
0       0.2     1.00      1.0     0.50      0.1
1       0.3     1.00      1.0     0.30      0.1
2       0.2     1.00      1.0     0.30      0.1
3       0.2     0.80      0.8     0.30      0.1
4       0.5     0.50      0.5     0.00      0.1
5       0.3     0.20      0.3     0.00      0.1
6       0.1     0.20      0.5     0.00      0.1
7       0.5     0.20      0.2     0.00      0.1
8       0.5     0.50      0.2     0.00      0.1
9       0.2     0.10      0.2     0.00      0.1
10      0.2     0.10      0.2     0.00      0.1
11      0.6     0.10      0.2     0.00      0.3
12      0.5     0.10      0.2     0.00      0.3
13      0.1     0.10      0.2     0.00      0.2
14      0.1     0.10      0.2     0.00      0.2
15      0.1     0.10      0.2     0.00      0.1
16      0.1     0.10      0.2     0.00      0.2
17      0.1     0.10      0.2     0.00      0.2
18      0.1     0.10      0.2     0.00      0.2
19      0.1     0.10      0.2     0.00  

In [27]:
fields = {}  # 필드를 저장할 빈 딕셔너리를 생성

# program_pref 각 필드에 대해 반복 (5개 열)
for f in program_prefs.columns:  
    # 각 필드에 대한 파일 경로 설정
    lattice_path = os.path.relpath('matrixs/' + f + '.csv')      
    try:
        # 배열 파일에서 데이터를 불러와서 fields 딕셔너리에 저장
        fields[f] = tg.lattice_from_csv(lattice_path)  
    except:
        # 파일이 없을 경우, 모든 값을 1로 설정한 기본 격자 데이터를 fields 딕셔너리에 저장
        fields[f] = copy.deepcopy(avail_lattice * 0 + 1)

# 'top_pre' 열의 인덱스 찾기 (5번째에 위치)
top_pre_index = df.columns.get_loc('top_pre')
# 'top_pre' 열 이후 모든 열을 포함하는 새로운 데이터프레임 생성 (프로그램 상관관계 matrix 찾기)
program_mtx = df.iloc[:, top_pre_index + 1:]
# 새로운 데이터프레임의 열 이름을 0부터 시작하는 숫자로 변경     (프로그램 번호입력 0~ ) 
program_mtx.columns = range(len(program_mtx.columns))

# program_prefs의 복사본 생성 (원본 데이터를 건드리지 않게)
program_prefs_copy = program_prefs.copy()
# program_prefs_copy에 'space_id' 열 추가 (원본 df의 인덱스 사용)
program_prefs_copy['space_id'] = df.index
# 인덱스를 재설정하여 연속적인 숫자 인덱스를 생성
program_prefs_copy.reset_index(drop=True, inplace=True)

# 최종 데이터프레임 생성
program_complete = program_prefs_copy

# 업데이트된 'program_prefs_copy' 데이터프레임 출력
print(program_prefs)


    sun_acc  ent_acc  str_acc  ung_pre  top_pre
0       0.2     1.00      1.0     0.50      0.1
1       0.3     1.00      1.0     0.30      0.1
2       0.2     1.00      1.0     0.30      0.1
3       0.2     0.80      0.8     0.30      0.1
4       0.5     0.50      0.5     0.00      0.1
5       0.3     0.20      0.3     0.00      0.1
6       0.1     0.20      0.5     0.00      0.1
7       0.5     0.20      0.2     0.00      0.1
8       0.5     0.50      0.2     0.00      0.1
9       0.2     0.10      0.2     0.00      0.1
10      0.2     0.10      0.2     0.00      0.1
11      0.6     0.10      0.2     0.00      0.3
12      0.5     0.10      0.2     0.00      0.3
13      0.1     0.10      0.2     0.00      0.2
14      0.1     0.10      0.2     0.00      0.2
15      0.1     0.10      0.2     0.00      0.1
16      0.1     0.10      0.2     0.00      0.2
17      0.1     0.10      0.2     0.00      0.2
18      0.1     0.10      0.2     0.00      0.2
19      0.1     0.10      0.2     0.00  

In [28]:
# 'space_name' 열을 딕셔너리로 변환하여 'space_list'에 저장
space_list = df['space_name'].to_dict()
# 'vox_amount' 열의 값을 가져와 'sizes_complete'에 저장
sizes_complete = df['vox_amount']
# 'agent_areas' 리스트 초기화
agent_areas = []
# 'sizes_complete'의 각 값에 대해 반복
for area in sizes_complete:
    # 각 값을 정수로 변환하고 반올림하여 'agent_areas'에 추가
    agent_areas.append(round(int(area)))

print(space_list)   # 변환된 딕셔너리를 출력
print(agent_areas)  # 'agent_areas' 리스트를 출력

{0: 'Lobby', 1: 'Branding Space', 2: 'External Contact', 3: 'Interview Room', 4: 'Personal Workspace', 5: 'Work Support Area', 6: 'Personal Locker Room', 7: 'Executive Space', 8: 'VIP Reception', 9: 'Secretary Room', 10: 'Pantry', 11: 'Executive Meeting', 12: 'Executive Offices', 13: 'Focus Rooms', 14: 'Project Rooms', 15: 'Conference Rooms', 16: '4-Person Meeting', 17: '6-Person Meeting', 18: '8-Person Meeting', 19: '12-Person Meeting', 20: '18-Person Meeting', 21: 'Idea Room', 22: 'Mail Room', 23: 'Oasis', 24: 'PT Practice Room', 25: 'Screening Room', 26: 'Studio', 27: 'Video Editing', 28: 'Plotter/Board Room', 29: 'Training Room', 30: 'Ino Kitchen', 31: 'Kitchen', 32: 'Ino Cafe', 33: 'Fitness Facilities', 34: 'Dressing/Shower', 35: 'Counseling Room', 36: 'Personal Rest Room', 37: 'Sleeping Room', 38: 'Nursing Room', 39: 'Work Lounge', 40: 'Town Hall', 41: 'Seminar Room', 42: 'Library', 43: 'Social Space', 44: 'Outdoor Terrace', 45: "Driver's Waiting", 46: 'Server Room', 47: 'Cleanin

In [29]:
# def find_neighbours_masked(lattice, stencil, loc):
#     """이웃을 찾는 함수"""
#     neigh_locs = np.argwhere(stencil) - stencil.origin + loc
#     neigh_filter = np.all(neigh_locs >= 0, axis=1) & np.all(neigh_locs < np.array(lattice.shape), axis=1)
#     return neigh_locs[neigh_filter]

# def initialize_agents(avail_lattice, program_prefs, fields):
#     """에이전트에 시작 위치 찾기 1단계 """
#     # 점유 배열(occupation lattice)를 초기화 (모든 값을 -1로 설정)
#     occ_lattice = avail_lattice * 0 - 1
#     # 에이전트 위치를 저장할 빈 리스트 생성
#     agn_locs = []

#     # 각 에이전트의 시작 위치를 설정 (program_prefs 행 반복)
#     for a_id, a_prefs in program_prefs.iterrows():
#         # 선호 배열(preference lattice) 초기화 (모든 값을 1로 설정)
#         pref_lattice = (avail_lattice * 0.1 + 1.0) * avail_lattice

#         # 각 필드에 대해 가중치를 적용하여 선호 격자를 업데이트
#         for f, w in a_prefs.items():
#             pref_lattice *= fields[f] ** w

#         # 선호 배열에서 최대값을 가지는 인덱스 선택
#         select_id = np.argmax(pref_lattice)
#         # 평평한(1차원) 배열의 특정 위치를 다시 원래의 다차원 배열의 위치로 변환
#         a_origin = np.unravel_index(select_id, avail_lattice.shape)
#         # z축 방향으로 최적 위치 탐색
#         for z in range(avail_lattice.shape[2]):
#             test_loc = (a_origin[0], a_origin[1], z)
#             if avail_lattice[test_loc] == 1:
#                 # 이웃을 찾고, 해당 위치가 사용 가능한지 확인
#                 neighbours = find_neighbours_masked(avail_lattice, stencil, test_loc)
#                 if all(occ_lattice[tuple(neigh)] == -1 for neigh in neighbours):
#                     a_origin = test_loc
#                     break

#        # 에이전트 위치 리스트에 추가
#         agn_locs.append(a_origin)
#         # 가용 배열에서 해당 위치를 0으로 설정 (이제 사용 불가능)
#         avail_lattice[a_origin] = 0
#         # 점유 배열에서 해당 위치를 에이전트 ID로 설정
#         occ_lattice[a_origin] = a_id

#     return occ_lattice, agn_locs

In [30]:
# """에이전트에 시작 위치 찾기 1단계 """

# agn_num = len(program_complete) 
# # 점유 배열(occupation lattice)를 초기화 (모든 값을 -1로 설정) 모든값을 0으로 곱한뒤 - 1 
# occ_lattice = avail_lattice * 0 - 1

# # 에이전트 위치를 저장할 빈 리스트 생성
# agn_locs = [None]*agn_num

# # 각 에이전트의 시작 위치를 설정 (program_prefs행 반복)
# for a_id, a_prefs in program_prefs.iterrows(): # a_id는 index (0,1,2,...) a_pref는 행의 데이터 
#     # 선호 배열(preference lattice) 초기화 모든값을 1로 설정 -> (가용 격자 값을 기반으로 설정)
#     pref_lattice = (avail_lattice * 0 + 1.0) * avail_lattice        

#     # 각 필드에 대해, 가중치를 적용하여 선호 격자를 업데이트
#     for f, w in a_prefs.items(): #f 는 항목의 이름(열의 이름:sun_acc,ent_acc) w는 선호도의 값 (0.5, ... )
#          # '*='는 곱하기 할당, 변수에 곱한 결과를 다시 변수에 할당
#          # 'w' 가중치로 에이전트의 선호도를 반영 [0.6**0.2] 이런식
#          pref_lattice *= fields[f] ** w     
#          # 선호 배열에서 최대값을 가지는 인덱스 선택
#          select_id = np.argmax(pref_lattice)
#           # 평평한(1차원) 배열의 특정 위치를 다시 원래의 다차원 배열의 위치로 변환
#          a_origin_1 = np.unravel_index(select_id, avail_lattice.shape)     
        
        
       
#     #     # 초기 위치를 계산된 위치로 설정 (z 축 값을 그대로 유지)
#          a_origin_2 = a_origin_1  # 또는 a_origin_1을 그대로 사용
#          a_origin_2 = (a_origin_1[0], a_origin_1[1], 1)

#     #     # 동일한 위치에 여러 에이전트가 배치되지 않도록 조정
#          for n in range(5):
#              if avail_lattice[a_origin_2] == 0:
#                  a_origin_2 = (a_origin_1[0], a_origin_1[1], a_origin_1[2])
                 

#     #     # 에이전트 위치 리스트에 추가
#          agn_locs.append([a_origin_2])
#     #     # 가용 배열에서 해당 위치를 0으로 설정 (이제 사용 불가능)
#          avail_lattice[a_origin_2] = 0
#          # 점유 배열에서 해당 위치를 에이전트 ID로 설정
#          occ_lattice[a_origin_2] = a_id

# print(agn_locs)

In [31]:
def initialize_agents(avail_lattice, program_prefs, fields):   
    """에이전트에 시작 위치 찾기 1단계 """
    # 점유 배열(occupation lattice)를 초기화 (모든 값을 -1로 설정) 모든값을 0으로 곱한뒤 - 1 
    agn_num = len(program_complete) 
    
    occ_lattice = avail_lattice * 0 - 1
    # 에이전트 위치를 저장할 빈 리스트 생성
    agn_locs = [None]*agn_num
    # 각 에이전트의 시작 위치를 설정 (program_prefs 행 반복)
    for a_id, a_prefs in program_prefs.iterrows(): # a_id는 index (0,1,2,...) a_pref는 행의 데이터 
        # 선호 배열(preference lattice) 초기화 모든값을 1로 설정 -> (가용 격자 값을 기반으로 설정)
        pref_lattice = (avail_lattice * 0 + 1.0) * avail_lattice

        # 각 필드에 대해, 가중치를 적용하여 선호 격자를 업데이트
        for f, w in a_prefs.items(): #f 는 항목의 이름(열의 이름:sun_acc,ent_acc) w는 선호도의 값 (0.5, ... )
            # '*='는 곱하기 할당, 변수에 곱한 결과를 다시 변수에 할당
            # 'w' 가중치로 에이전트의 선호도를 반영 [0.6**0.2] 이런식
            pref_lattice *= fields[f] ** w

        # 선호 배열에서 최대값을 가지는 인덱스 선택
        select_id = np.argmax(pref_lattice)
        # 평평한(1차원) 배열의 특정 위치를 다시 원래의 다차원 배열의 위치로 변환
        a_origin_1 = np.unravel_index(select_id, avail_lattice.shape)
        
       
        # 초기 위치를 계산된 위치로 설정 (z 축 값을 그대로 유지)
        a_origin_2 = a_origin_1  # 또는 a_origin_1을 그대로 사용
        a_origin_2 = (a_origin_1[0], a_origin_1[1], 1)

        # 동일한 위치에 여러 에이전트가 배치되지 않도록 조정
        for n in range(5):
            if avail_lattice[a_origin_2] == 0:
                a_origin_2 = (a_origin_1[0], a_origin_1[1], a_origin_1[2])

        # 에이전트 위치 리스트에 추가
        agn_locs.append([a_origin_2])
        # 가용 배열에서 해당 위치를 0으로 설정 (이제 사용 불가능)
        avail_lattice[a_origin_2] = 0
        # 점유 배열에서 해당 위치를 에이전트 ID로 설정
        occ_lattice[a_origin_2] = a_id

    return occ_lattice, agn_locs



In [32]:
def find_optimal_location_for_agent(agent_id, agent_prefs, avail_lattice, fields, stencil, occ_lattice, max_attempts=100):
    """에이전트에 대한 최적의 시작 위치 찾기 (2단계)"""
    
    # 가용 배열의 인덱스들을 배열로 변환 
    # np.where() 배열에서 'True'인 요소의 인덱스를 반환 = avail_lattice 배열에서 True 값을 가지는 위치 / 반환값은 세 개의 1차원 배열, 각각 x, y, z 축의 인덱스를 나타냅니다.
    # np.array() 반환된 인덱스 배열을 하나의 2차원 배열로 (행렬 형태)
    avail_index = np.array(np.where(avail_lattice)).T 
    # 에이전트 평가 배열을 1로 초기화 avail_index의 배열의 길이로, np.ones 그 길이만큼 모든요소가 1인 배열 생성
    a_eval = np.ones(len(avail_index))

    # 각 선호도 열에 대해 반복
    for f in program_prefs.columns:
        # 현재 선호도 열에 해당하는 필드 값들을 가져옴
        vals = fields[f][avail_index[:, 0], avail_index[:, 1], avail_index[:, 2]]
        # 값이 무한대인 경우 0으로 설정하고, 선호도 가중치를 적용
        a_weighted_vals = np.where(np.isinf(vals), 0, vals ** agent_prefs[f])
        # 평가 배열에 가중치 적용된 값을 곱함
        a_eval *= a_weighted_vals

    # 평가 배열을 기준으로 인덱스를 내림차순으로 정렬
    sorted_indices = np.argsort(-a_eval)
    # 평가 배열을 기준으로 인덱스를 내림차순으로 정렬
    sorted_indices = np.argsort(-a_eval)
    attempts = 0

    for selected_int in sorted_indices:
        if attempts >= max_attempts:
            break  # 최대 시도 횟수를 초과하면 종료

        selected_ind = avail_index[selected_int]

        while True:
            fns = avail_lattice.find_neighbours_masked(stencil, loc=selected_ind)
            blocked = 0
            for n in fns:
                neigh_3d_id = np.unravel_index(n, avail_lattice.shape)
                if occ_lattice[neigh_3d_id] != -1:
                    blocked += 1

            if blocked >= 2:
                selected_int += 1
                if selected_int >= len(sorted_indices):
                    break  # 선택할 수 있는 인덱스를 초과하면 종료
                selected_ind = avail_index[sorted_indices[selected_int]]
            else:
                print(f"Optimal location for agent {agent_id} found at {selected_ind}")
                return selected_ind

        attempts += 1

    # 적합한 위치를 찾지 못한 경우
    print(f"No suitable location found for agent {agent_id} after {attempts} attempts")    
    return None

In [33]:
# 초기 시작 위치 1단계
occ_lattice, agn_locs = initialize_agents(avail_lattice, program_prefs, fields)
# 각 에이전트의 최적 위치를 찾기
for a_id, a_prefs in program_prefs.iterrows():
    # 각 에이전트에 대해 최적의 위치를 찾음
    optimal_location = find_optimal_location_for_agent(a_id, a_prefs, avail_lattice, fields, stencil, occ_lattice)
    if optimal_location is not None:
        print(f"Agent {space_list[a_id]} optimal location: {optimal_location}")
    else:
        print(f"Agent {space_list[a_id]} has no suitable location.")

Optimal location for agent 0 found at [32  8  5]
Agent Lobby optimal location: [32  8  5]
Optimal location for agent 1 found at [32  8  5]
Agent Branding Space optimal location: [32  8  5]
Optimal location for agent 2 found at [32  8  5]
Agent External Contact optimal location: [32  8  5]
Optimal location for agent 3 found at [32  8  5]
Agent Interview Room optimal location: [32  8  5]
Optimal location for agent 4 found at [46 10  8]
Agent Personal Workspace optimal location: [46 10  8]
Optimal location for agent 5 found at [46 10  8]
Agent Work Support Area optimal location: [46 10  8]
Optimal location for agent 6 found at [18 17  3]
Agent Personal Locker Room optimal location: [18 17  3]
Optimal location for agent 7 found at [46 12 10]
Agent Executive Space optimal location: [46 12 10]
Optimal location for agent 8 found at [46 10  8]
Agent VIP Reception optimal location: [46 10  8]
Optimal location for agent 9 found at [ 5 32  0]
Agent Secretary Room optimal location: [ 5 32  0]
Opti

In [34]:
from matplotlib.colors import ListedColormap
import pyvista as pv

# Visualization setup
p = pv.Plotter(notebook=True)

# Set the grid dimensions
grid = pv.ImageData()
grid.dimensions = np.array(occ_lattice.shape) + 1
# grid.origin과 grid.spacing 설정에 avail_lattice의 속성 사용
grid.origin = avail_lattice.minbound - avail_lattice.unit * 0.5
grid.spacing = avail_lattice.unit

# Adding the bounding box wireframe
p.add_mesh(grid.outline(), color="grey", label="Domain")
p.add_axes()
p.show_bounds(grid="back", location="back", color="#777777")

# Add the data values to the cell data
grid.cell_data["Agents"] = occ_lattice.flatten(order="F").astype(int)  # Flatten the array!

# Filtering the voxels
agn_num = len(program_prefs)
threshed = grid.threshold([-0.1, agn_num - 0.9])

colors = [
    "#FFB6C1", "#FFC0CB", "#FF69B4", "#FF1493",  # 분홍색 계열
    "#87CEEB", "#00BFFF", "#1E90FF",  # 파란색 계열
    "#98FB98", "#00FF7F", "#32CD32", "#3CB371", "#2E8B57", "#228B22",  # 녹색 계열
    "#FFD700", "#FFA500", "#FF8C00", "#FF7F50", "#FF6347", "#FF4500", "#FF0000", "#DC143C", "#B22222",  # 노란색 계열
    "#BA55D3", "#9932CC", "#9400D3", "#8A2BE2", "#9370DB", "#7B68EE", "#6A5ACD",  # 자주색 계열
    "#FFE4B5", "#FFDAB9", "#FFDEAD",  # 베이지 계열
    "#F0E68C", "#EEE8AA", "#F5DEB3", "#DEB887", "#D2B48C", "#BC8F8F",  # 갈색 계열
    "#B0C4DE", "#ADD8E6", "#B0E0E6", "#AFEEEE", "#00CED1",  # 연한 파란색 계열
    "#696969",  # 회색 계열
    "#708090",  # 슬레이트 그레이 계열
    "#4682B4", "#5F9EA0", "#6495ED", "#7B68EE", "#87CEFA", "#87CEFA"  # 추가 파란색 계열
]

custom_cmap = ListedColormap(colors)
# Scalar bar 설정
sargs = dict(
    shadow=True,
    n_labels=agn_num,
    italic=False,
    fmt=" %.0f",
    font_family="arial",
    height=0.6,
    vertical=True,

    title_font_size=10,  # 제목 폰트 크기 설정
    label_font_size=6   # 라벨 폰트 크기 설정
)
annotations = {i: name for i, name in enumerate(space_list.values())}
# Adding the voxels with the custom colormap and scalar bar arguments
p.add_mesh(threshed, name='sphere', show_edges=True, opacity=1.0, show_scalar_bar=True, annotations=annotations, scalar_bar_args=sargs, cmap=custom_cmap)
street_pc.fast_notebook_vis(p)
p.add_mesh(tri_to_pv(context_mesh), color='#aaaaaa', opacity=0.3)
p.add_mesh(entrance_pc, color='red', opacity=1.0)
#envelope_lattice.fast_vis(p)
# Show the plot
p.show(jupyter_backend="trame", return_viewer=True)  # 시각화 표시



Widget(value='<iframe src="http://localhost:64214/index.html?ui=P_0x21352ede590_5&reconnect=auto" class="pyvis…

In [35]:
from matplotlib.colors import ListedColormap

# PyVista 플로터 초기화
p = pv.Plotter(notebook=True)

# 그리드 설정
#base_lattice = frames[0]
grid = pv.ImageData()
grid.dimensions = np.array(occ_lattice.shape) + 1
# grid.origin과 grid.spacing 설정에 avail_lattice의 속성 사용
grid.origin = avail_lattice.minbound - avail_lattice.unit * 0.5
grid.spacing = avail_lattice.unit

# 데이터 할당
grid.cell_data["Agents"] = occ_lattice.flatten(order="F").astype(int)

# 필터링
agn_num = len(program_prefs)
threshed = grid.threshold([-0.1, agn_num - 0.9])

# 초기 컬러맵 설정
initial_colors = ["#808080" if i == 0 else "#FF0000" for i in range(len(program_prefs))]
custom_cmap = ListedColormap(initial_colors)

# 스칼라 바 설정
annotations = {i: name for i, name in enumerate(space_list.values())}
sargs = {
    "shadow": True,
    "n_labels": len(program_prefs),
    "italic": False,
    "fmt": " %.0f",
    "font_family": "arial",
    "height": 1,
    "vertical": True,
    "title_font_size": 10,
    "label_font_size": 6
}

# 메시 추가
opacity_map = np.full(threshed.n_cells, 0.1)  # 초기 투명도 설정
threshed.cell_data['opacity'] = opacity_map
mesh = p.add_mesh(threshed, scalars='opacity', cmap=custom_cmap, show_scalar_bar=True, scalar_bar_args=sargs, show_edges=True, opacity=1)

# 텍스트 추가
text_actor = p.add_text("Selected Program: None", position='upper_right', font_size=12, color='black')

# 슬라이더 콜백 함수
def slider_callback(value):
    selected_program = int(value)
    mask = threshed.cell_data['Agents'] == selected_program
    opacity = np.full(threshed.n_cells, 0.1)  # 기본 투명도 설정
    opacity[mask] = 1.0  # 선택된 프로그램에 높은 투명도 부여

    # 투명도 업데이트
    threshed.cell_data['opacity'] = opacity
    
    # 텍스트 업데이트
    text_actor.SetText(0, f"Selected Program: {annotations[selected_program]}")
    p.render()

# 슬라이더 위젯 추가
p.add_slider_widget(slider_callback, [0, len(program_prefs) - 1], value=0, title='Program Index')
p.add_mesh(threshed, name='sphere', show_edges=True, opacity=1.0, show_scalar_bar=True, annotations=annotations, scalar_bar_args=sargs, cmap=custom_cmap)

p.add_mesh(tri_to_pv(context_mesh), color='#aaaaaa', opacity=0.3)
p.add_mesh(entrance_pc, color='red', opacity=1.0)
# 시각화 표시
p.show(jupyter_backend="trame", return_viewer=True)

Widget(value='<iframe src="http://localhost:64214/index.html?ui=P_0x213669a0fd0_6&reconnect=auto" class="pyvis…